# Below is a step-by-step example that how the pharma_subgraph KG used for SQID is built

Set up environment variables with location of the input files

In [3]:
%env kgtk_files=kgtk_files/
%env ttl_files=ttl_files/
%env ES_DATA_FOLDER_PATH = flask/es/es_data/

env: kgtk_files=kgtk_files/
env: ttl_files=ttl_files/
env: ES_DATA_FOLDER_PATH=flask/es/es_data/


## Building Wikidata triples and Mediawiki json files

pharma_subgraph.tsv link: https://drive.google.com/file/d/11oAhqCNR8vAIKXv0IhuIcy-L6NuDFxKF/view?usp=sharing

### Generate Wikidata Triples (under sqid/ttl_files) from KGTK TSV (pharma_subgraph) -- kgtk generate_wikidata_triples

In [4]:
!cat kgtk_files/pharma_subgraph.tsv \
| kgtk generate_wikidata_triples -pf flask/flask_data/wikidata_properties.tsv -w yes -pd yes > pharma_subgraph.ttl --debug

In [8]:
!mv *.ttl ttl_files/

### Generate Mediawiki JSON (under sqid/flask/es/es_data/) from KGTK TSV -- kgtk generate-mediawiki-jsons

In [5]:
!cp kgtk_files/pharma_subgraph.tsv ES_DATA_FOLDER_PATH/pharma_subgraph.tsv
 
!cat ES_DATA_FOLDER_PATH/pharma_subgraph.tsv \
| kgtk generate-mediawiki-jsons -pf flask/flask_data/wikidata_properties.tsv -w yes -pd yes

In [6]:
!mv *.jsonl ES_DATA_FOLDER_PATH/

In [6]:
!cd ES_DATA_FOLDER_PATH/ && ls | wc -l

/bin/sh: line 0: cd: ES_DATA_FOLDER_PATH/: No such file or directory


## Load trile file into Blazegraph

Given triple files loaded into Blazegraph

## Load json file into ES

### Install all required libraries

In [ ]:
!python3 -m venv sqid-env
!source sqid-env/bin/activate

!cd flask && pip install -r requirement.txt
!npm install .

### Modify Configuration file (Global configure)
Parameters:
- HOST = "localhost"
- SQID_PORT = 8052
- SPARQL_PORT = 10002
- FLASK_PORT = 5556
- ES_INDEX = "kgtk_files"
- ELASTICSEARCH_PORT = 9200
- ES_DATA_FOLDER_PATH = "/home/mingyuet/sqid/flask/es/es_data"

In [4]:
!head -n 15 global_config.py

import argparse

parser = argparse.ArgumentParser(description='Enter congiguration parameters')
parser.add_argument('--is_generate', type=bool, help='is_generate', default=False)
args = parser.parse_args()

HOST = "localhost"
SQID_PORT = 8052
SPARQL_PORT = 11102
FLASK_PORT = 5556
ES_INDEX = "kgtk_files"
ELASTICSEARCH_PORT = 9200
is_generate = args.is_generate

# Flask application config


### Create Index for ES

In [20]:
# index name is defined in global_config file
!cd flask/es/ && python create_index.py

### import index data from es_data (mediawiki json files)

In [21]:
# mediawiki json files path(es_data) is defined in global_config file
!cd flask/es/ && python import_data.py

100%|███████████████████████████████████████████| 90/90 [00:08<00:00, 10.87it/s]


### Run Global Configure File to generate endpoints for SQID and ES

In [24]:
!python global_config.py --is_generate True

### Run ES index system using Flask

and

### Run SQID through npm run serve (development enviorment)

In [5]:
import subprocess
out = subprocess.call("ls -l", shell=True)
out = subprocess.call("cd ..", shell=True)